In [ ]:
#based on Qiskit Textbook's tutorial
from qiskit.tools.jupyter import *
from qiskit import *
IBMQ.load_account()
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')

In [ ]:
#ONLY ARMONK SUPPORTS OPEN_PULSE
backend = provider.get_backend('ibmq_armonk')

In [ ]:
"""
Experiment 1 - measure T1 (characteristic amplitude dampening time)
using OpenQASM and IDENTITY GATES to create the delays
"""
#get this from Yael's backend properties getter (units of us)
gate_time = 0.14222222222


# Note that it is possible to measure several qubits in parallel
qubits = [0]

In [ ]:

from qiskit.ignis.characterization.coherence import t1_circuits

num_of_gates = np.linspace(10, 5000, 50, dtype='int')


#INSERT GATE_TIME FROM THE BACKEND CONFIG OBJECT
t1_circs, t1_xdata = t1_circuits(num_of_gates, gate_time, qubits)
shots = 1024

# Let the simulator simulate the following times for qubits 0 and 2:

t1_backend_result = qiskit.execute(t1_circs, backend, shots=shots,
                                   
                                   optimization_level=0).result()

In [ ]:
from qiskit.ignis.characterization.coherence import T1Fitter

plt.figure(figsize=(15, 6))

#this fits to prob_1(t) = Aexp(-t/T) + B UNITS IN MICRO S
t1_fit = T1Fitter(t1_backend_result, t1_xdata, qubits,
                  fit_p0=[1, 200, 0],
                  fit_bounds=([0, 0, -1], [2, 500, 1]))
print(t1_fit.time())
print(t1_fit.time_err())
print(t1_fit.params)
print(t1_fit.params_err)

for i in range(1):
    ax = plt.subplot(1, 2, i+1)
    t1_fit.plot(i, ax=ax)
plt.show()


In [ ]:
"""
Experiment 2 - measure errors of Pauli gates
"""
